# Forth Addition

The forth addition problem is another example used in (Manhaeve et al. 2018) to illustrate DeepProbLog’s ability to do both logical reasoning and deep learning. The task is, given

(i) a pair of list of digits, e.g.,

    [8, 4] and [5, 2]

(ii) a carry-in, e.g., 

    1

and (iii) a list of digits denoting their addition, e.g., (84+52+1=137)

    [1, 3, 7]

train the neural network m1 to predict the result at each location (e.g., 7 at the unit digit) and train the neural network m2 to predict the carry-out at each location (e.g., 0 at the unit digit) using the 2 digits and the carry-in (e.g., 4, 2, and 1 at the unit digit) at that location as input. Note that the 3 numbers 4, 2, and 1 are turned into the following vector of length 30 before feeding into the neural networks.

    [  00001 00000
       00100 00000
       01000 00000  ]

In the above example, the label for neural network m1 is 7 while the label for neural network m2 is 0.

## Data Format
Below are 20 samples of data following the data format used in DeepProbLog.

In [1]:
from dataGen import create_data_sample, format_dataList, format_observations
for i in range(20):
    print(create_data_sample()[0])

add([6,1],[8,8],0,[1,4,9]).
add([1,1],[0,8],1,[0,2,0]).
add([4,7],[9,5],0,[1,4,2]).
add([9,9],[6,3],0,[1,6,2]).
add([5,1],[4,0],1,[0,9,2]).
add([2,4],[9,3],0,[1,1,7]).
add([4,9],[4,5],0,[0,9,4]).
add([7,9],[5,3],1,[1,3,3]).
add([6,7],[1,3],0,[0,8,0]).
add([5,9],[1,7],1,[0,7,7]).
add([4,7],[5,7],0,[1,0,4]).
add([3,7],[0,3],1,[0,4,1]).
add([4,7],[6,8],0,[1,1,5]).
add([4,9],[5,7],1,[1,0,7]).
add([8,3],[4,6],1,[1,3,0]).
add([4,1],[1,3],0,[0,5,4]).
add([1,7],[0,7],1,[0,2,5]).
add([3,6],[1,9],0,[0,5,5]).
add([0,0],[5,1],0,[0,5,1]).
add([6,8],[1,0],1,[0,7,9]).


Let's consider the following sample data.

In [2]:
obs,str_list = create_data_sample()
print('Sample data:')
print(obs)
print('\nThis sample data will be turned into the following dictionary (an element in dataList)')
print(format_dataList(obs,str_list))
print('\nThis sample data will be also turned into the following string (an element in obsList)')
print(format_observations(obs,str_list))

Sample data:
add([6,7],[8,5],0,[1,5,2]).

This sample data will be turned into the following dictionary (an element in dataList)
{'num1_0,num2_0,0': tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_0,num2_0,1': tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_1,num2_1,0': tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), 'num1_1,num2_1,1': tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])}

This sample data will be also turned into the following string (an element in obsList)

    :-not carry(0,0).
    :-not carry(2,1).
    :-not result(1,5).
    :-not result(0,2).
    


## Imports

In [3]:
import sys
sys.path.append("../../")
import time

import torch

from dataGen import dataList, obsList, add_test_dataloader, carry_test_dataloader
from dlpmln import DeepLPMLN
from network import FC

## DeepLPMLN Program

In [4]:
dprogram='''
num1(1,num1_1).
num1(0,num1_0).
num2(1,num2_1).
num2(0,num2_0).
1{carry(0,Carry): Carry=0..1}1.

nn(m1(A, B, Carry, 1), result, [0,1,2,3,4,5,6,7,8,9]) :- num1(P,A), num2(P,B), Carry=0..1.
nn(m2(A, B, Carry, 1), carry, [0,1]) :- num1(P,A), num2(P,B), Carry=0..1.


%The first argument of carry/2 and result/2 is the location and the second argument is the value. 

result(P,X) :- num1(P, A), num2(P, B), carry(P, Carry), result(A,B,Carry,0,X).
carry(P+1,X) :- num1(P, A), num2(P, B), carry(P, Carry), carry(A,B,Carry,0,X).
'''

## Neural Network Instantiation
- Instantiate neural networks.
- Define nnMapping: a dictionary that maps neural network names (i.e., strings) to the neural network objects (i.e., torch.nn.Module object)
- Define optimizers: a dictionary that specifies the optimizer for each network (we use the Adam optimizer here).

In [5]:
m1 = FC(30,25,10) # network for adding the numbers
m2 = FC(30,5,2)   # network for finding the carry out 

nnMapping = {'m1':m1, 'm2':m2}
optimizers = {'m1':torch.optim.Adam(m1.parameters(), lr=0.01),'m2':torch.optim.Adam(m2.parameters(), lr=0.01)}

## Create DeepLPMLN Object

In [6]:
dlpmlnObj = DeepLPMLN(dprogram, nnMapping, optimizers)

## Training and Testing 

In [7]:
startTime = time.time()
for i in range(5):
    print('Epoch {}...'.format(i+1))
    time1 = time.time()
    dlpmlnObj.learn(dataList=dataList, obsList=obsList, epoch=1)
    time2 = time.time()
    dlpmlnObj.testNN("m1", add_test_dataloader) #test m1 network
    dlpmlnObj.testNN("m2", carry_test_dataloader) #test m2 network
    print("--- train time: %s seconds ---" % (time2 - time1))
    print("--- test time: %s seconds ---" % (time.time() - time2))
    print('--- total time from beginning: %s minutes ---' % int((time.time() - startTime)/60) )

Epoch 1...
Test Accuracy on NN Only for m1: 43%
Test Accuracy on NN Only for m2: 95%
--- train time: 139.4350049495697 seconds ---
--- test time: 0.011136054992675781 seconds ---
--- total time from beginning: 2 minutes ---
Epoch 2...
Test Accuracy on NN Only for m1: 85%
Test Accuracy on NN Only for m2: 98%
--- train time: 138.66707825660706 seconds ---
--- test time: 0.008195638656616211 seconds ---
--- total time from beginning: 4 minutes ---
Epoch 3...
Test Accuracy on NN Only for m1: 97%
Test Accuracy on NN Only for m2: 100%
--- train time: 137.17292094230652 seconds ---
--- test time: 0.0081939697265625 seconds ---
--- total time from beginning: 6 minutes ---
Epoch 4...
Test Accuracy on NN Only for m1: 99%
Test Accuracy on NN Only for m2: 100%
--- train time: 136.48796391487122 seconds ---
--- test time: 0.008369922637939453 seconds ---
--- total time from beginning: 9 minutes ---
Epoch 5...
Test Accuracy on NN Only for m1: 99%
Test Accuracy on NN Only for m2: 100%
--- train time: